In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE

# reading the ratings data
ratings = pd.read_csv('ml-latest-small/ratings.csv', sep=",")

ratings_pivot = pd.pivot_table(ratings[['userId', 'movieId', 'rating']], 
                               values='rating', index='userId', columns='movieId' ).fillna(0)


# creating train and test sets
X_train, X_test = train_test_split(ratings_pivot, train_size=0.8)
# print(X_train)


In [18]:
# ratings.to_csv("ratings2.csv", index=False)

In [5]:
# Deciding how many nodes each layer should have - Depending on the dataset's size
movies_size = 9724 #15159 #58099
n_nodes_inpl = movies_size
n_nodes_hl1  = 256
n_nodes_outl = movies_size

# first hidden layer has 9723*256 weights and 256 biases
hidden_1_layer_vals = {'weights':tf.Variable(tf.random_normal([n_nodes_inpl+1, n_nodes_hl1]))}
output_layer_vals = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1+1, n_nodes_outl]))}

input_layer = tf.placeholder('float', [None, movies_size])

input_layer_const = tf.fill([tf.shape(input_layer)[0], 1], 1.0)
input_layer_concat = tf.concat([input_layer, input_layer_const], 1)

layer_1 = tf.nn.sigmoid(tf.matmul(input_layer_concat, hidden_1_layer_vals['weights']))

layer1_const = tf.fill( [tf.shape(layer_1)[0], 1], 1.0)
layer_concat =  tf.concat([layer_1, layer1_const], 1)

output_layer = tf.matmul(layer_concat, output_layer_vals['weights'])
output_true = tf.placeholder('float', [None, movies_size])
meansq = tf.reduce_mean(tf.square(output_layer - output_true))

learn_rate = 0.1   # learning rate
optimizer = tf.train.AdagradOptimizer(learn_rate).minimize(meansq)

# initializing variables and starting the session
init = tf.global_variables_initializer()
sess = tf.Session()
writer = tf.summary.FileWriter('./graphs', tf.get_default_graph())
writer.close()
sess.run(init)    


## Autoencoder with one hidden layer

![autoencoder with one layer](Images/autoencoders-1layer.png)

In [13]:
batch_size = 100
hm_epochs = 10000
tot_users = X_train.shape[0]
# print(tot_users)

for epoch in range(hm_epochs):
    epoch_loss = 0    # initializing error as 0
    for i in range(int(tot_users/batch_size)):
        # print(epoch_x)
        epoch_x = X_train[ i*batch_size : (i+1)*batch_size ]
        _, c = sess.run([optimizer, meansq],feed_dict={input_layer: epoch_x, output_true: epoch_x})
        epoch_loss += c
        
    output_train = sess.run(output_layer, feed_dict={input_layer:X_train})
    output_test = sess.run(output_layer, feed_dict={input_layer:X_test})
    print('MSE train', MSE(output_train, X_train),'MSE test', MSE(output_test, X_test))
    print('MAE train', MAE(output_train, X_train),'MAE test', MAE(output_test, X_test))
    print('Epoch', epoch, '/', hm_epochs, 'loss:',epoch_loss)

MSE train 6.2658078166941795 MSE test 12.77324684168542
MAE train 1.739454665773725 MAE test 2.5067948726481464
Epoch 0 / 10000 loss: 16.921318531036377
MSE train 6.2656228215758825 MSE test 12.77315053670004
MAE train 1.7394266726656857 MAE test 2.506788905793955
Epoch 1 / 10000 loss: 16.92067265510559
MSE train 6.265449999953294 MSE test 12.772911752894911
MAE train 1.7394012218811732 MAE test 2.506760795879351
Epoch 2 / 10000 loss: 16.91995644569397
MSE train 6.265265039930655 MSE test 12.77281569200617
MAE train 1.7393732348855884 MAE test 2.5067548564032105
Epoch 3 / 10000 loss: 16.91930890083313
MSE train 6.26509224530624 MSE test 12.772576907063376
MAE train 1.7393477901656602 MAE test 2.5067267572478404
Epoch 4 / 10000 loss: 16.918593883514404
MSE train 6.264907233629095 MSE test 12.772480950493577
MAE train 1.7393197957526845 MAE test 2.506720828140125
Epoch 5 / 10000 loss: 16.917946338653564
MSE train 6.2647345085592185 MSE test 12.772242273577703
MAE train 1.7392943600736495

MSE train 6.256150988467553 MSE test 12.764309784832307
MAE train 1.7380118359572154 MAE test 2.5058869229442813
Epoch 54 / 10000 loss: 16.884517192840576
MSE train 6.25596591242861 MSE test 12.764218397909517
MAE train 1.7379837753101453 MAE test 2.5058815149715374
Epoch 55 / 10000 loss: 16.883867979049683
MSE train 6.2557931219917915 MSE test 12.763984674540872
MAE train 1.737958357950136 MAE test 2.5058539550166294
Epoch 56 / 10000 loss: 16.88315200805664
MSE train 6.2556080149003925 MSE test 12.76389347486571
MAE train 1.7379302894178543 MAE test 2.5058485692934953
Epoch 57 / 10000 loss: 16.882503032684326
MSE train 6.255435097577902 MSE test 12.763660251846296
MAE train 1.7379048597242601 MAE test 2.505821070275472
Epoch 58 / 10000 loss: 16.881786346435547
MSE train 6.255249994285277 MSE test 12.76356910190652
MAE train 1.7378767901559997 MAE test 2.505815682254052
Epoch 59 / 10000 loss: 16.881137132644653
MSE train 6.255077058734904 MSE test 12.763336331357221
MAE train 1.7378513

MSE train 6.246490570091193 MSE test 12.755658199561882
MAE train 1.7365690233064643 MAE test 2.5050122740791565
Epoch 108 / 10000 loss: 16.847585678100586
MSE train 6.2463066573379695 MSE test 12.75556855600468
MAE train 1.736541084916559 MAE test 2.505007177447167
Epoch 109 / 10000 loss: 16.846939086914062
MSE train 6.246135038349135 MSE test 12.755337334572413
MAE train 1.7365158194619423 MAE test 2.504979988675323
Epoch 110 / 10000 loss: 16.846225023269653
MSE train 6.245951173956051 MSE test 12.755247379174053
MAE train 1.7364878851078416 MAE test 2.504974868119119
Epoch 111 / 10000 loss: 16.845579385757446
MSE train 6.2457796889253565 MSE test 12.75501604462168
MAE train 1.736462633720655 MAE test 2.504947675339845
Epoch 112 / 10000 loss: 16.844865322113037
MSE train 6.2455958579242115 MSE test 12.754926171504442
MAE train 1.7364346935668944 MAE test 2.5049425848016766
Epoch 113 / 10000 loss: 16.84421968460083
MSE train 6.2454243918088705 MSE test 12.75469479118477
MAE train 1.73

MSE train 6.237088146641959 MSE test 12.747144363185413
MAE train 1.735158583592315 MAE test 2.5041602417500037
Epoch 161 / 10000 loss: 16.81166386604309
MSE train 6.236917396270545 MSE test 12.7469078707209
MAE train 1.73513338829951 MAE test 2.504132419258012
Epoch 162 / 10000 loss: 16.810952186584473
MSE train 6.2367343252545435 MSE test 12.746812148896502
MAE train 1.7351054548901808 MAE test 2.5041266377940765
Epoch 163 / 10000 loss: 16.81031107902527
MSE train 6.2365635427653725 MSE test 12.74657525976953
MAE train 1.7350802536678471 MAE test 2.50409877685125
Epoch 164 / 10000 loss: 16.809600830078125
MSE train 6.236380473544608 MSE test 12.746478997170849
MAE train 1.7350523188241112 MAE test 2.5040928996183025
Epoch 165 / 10000 loss: 16.808958053588867
MSE train 6.236209753244224 MSE test 12.746241518060454
MAE train 1.7350271309698662 MAE test 2.5040649582668113
Epoch 166 / 10000 loss: 16.808248043060303
MSE train 6.236026670491804 MSE test 12.746144772359353
MAE train 1.73499

MSE train 6.227690341195859 MSE test 12.737973443086599
MAE train 1.7337488640986984 MAE test 2.503221281593524
Epoch 214 / 10000 loss: 16.775813341140747
MSE train 6.22750538161526 MSE test 12.737867968265926
MAE train 1.7337206654348791 MAE test 2.5032142939122353
Epoch 215 / 10000 loss: 16.775169372558594
MSE train 6.227332698202568 MSE test 12.737621804116944
MAE train 1.7336952414182751 MAE test 2.5031852630624307
Epoch 216 / 10000 loss: 16.77445697784424
MSE train 6.227147562551628 MSE test 12.737516089157573
MAE train 1.7336670174428197 MAE test 2.503178263122953
Epoch 217 / 10000 loss: 16.77381134033203
MSE train 6.226974698302374 MSE test 12.737269770251029
MAE train 1.733641566347432 MAE test 2.503149211746621
Epoch 218 / 10000 loss: 16.77309823036194
MSE train 6.226789258120087 MSE test 12.737163989743012
MAE train 1.7336132997400626 MAE test 2.503142192578707
Epoch 219 / 10000 loss: 16.772452116012573
MSE train 6.22661614452114 MSE test 12.736917484705604
MAE train 1.733587

...

MAE train 1.5167036656058315 MAE test 2.3508864591968357
Epoch 9773 / 10000 loss: 11.815123319625854
MSE train 4.899748213770757 MSE test 11.308063831336389
MAE train 1.516688424059673 MAE test 2.3508523270066695
Epoch 9774 / 10000 loss: 11.814711809158325
MSE train 4.899646451289072 MSE test 11.30808292880635
MAE train 1.5166672253448696 MAE test 2.3508601300459255
Epoch 9775 / 10000 loss: 11.81440806388855
MSE train 4.899541836347436 MSE test 11.307845004833275
MAE train 1.516651484655855 MAE test 2.350829041480466
Epoch 9776 / 10000 loss: 11.813971519470215
MSE train 4.899449100397675 MSE test 11.307851138106228
MAE train 1.5166329172661344 MAE test 2.350835072308687
Epoch 9777 / 10000 loss: 11.813647747039795
MSE train 4.8993456056892075 MSE test 11.307579861638503
MAE train 1.5166180430132998 MAE test 2.350799520402162
Epoch 9778 / 10000 loss: 11.813255548477173
MSE train 4.899239017767621 MSE test 11.307602984092178
MAE train 1.5165954474538044 MAE test 2.350807900660884
Epoch 97

MSE train 4.894511585313263 MSE test 11.301913532044503
MAE train 1.5157694561244177 MAE test 2.350184834315987
Epoch 9826 / 10000 loss: 11.79580283164978
MSE train 4.894412096446229 MSE test 11.301932858510852
MAE train 1.5157488320747858 MAE test 2.350192664972436
Epoch 9827 / 10000 loss: 11.795500040054321
MSE train 4.894308451790482 MSE test 11.301688851753925
MAE train 1.5157334405321279 MAE test 2.350160765179531
Epoch 9828 / 10000 loss: 11.795072555541992
MSE train 4.894213993843571 MSE test 11.30170097032109
MAE train 1.5157142631260891 MAE test 2.350167610465155
Epoch 9829 / 10000 loss: 11.794757843017578
MSE train 4.894110679958888 MSE test 11.30143843828238
MAE train 1.515699261820287 MAE test 2.3501332320746484
Epoch 9830 / 10000 loss: 11.794355154037476
MSE train 4.894008393960235 MSE test 11.301459436617174
MAE train 1.5156778307498702 MAE test 2.3501413024895457
Epoch 9831 / 10000 loss: 11.794057369232178
MSE train 4.893904384395976 MSE test 11.301228941687786
MAE train 

MSE train 4.889187343300051 MSE test 11.295810299540118
MAE train 1.5148312137966446 MAE test 2.349529839839685
Epoch 9879 / 10000 loss: 11.776681184768677
MSE train 4.889084211256447 MSE test 11.295603898449603
MAE train 1.5148151796359017 MAE test 2.3495029615244745
Epoch 9880 / 10000 loss: 11.776219844818115
MSE train 4.888992884904237 MSE test 11.295582133556573
MAE train 1.5147975686641983 MAE test 2.3495053198186384
Epoch 9881 / 10000 loss: 11.775880336761475
MSE train 4.888890623874143 MSE test 11.295326973718293
MAE train 1.5147826959355009 MAE test 2.3494719486092364
Epoch 9882 / 10000 loss: 11.775513648986816
MSE train 4.888790386961012 MSE test 11.295347782459105
MAE train 1.514761819993697 MAE test 2.3494800219328273
Epoch 9883 / 10000 loss: 11.775213479995728
MSE train 4.888686833356763 MSE test 11.295109129537572
MAE train 1.5147463384796258 MAE test 2.34944888407478
Epoch 9884 / 10000 loss: 11.77478313446045
MSE train 4.88859408344178 MSE test 11.29511884140168
MAE train

MAE train 1.5139000016275186 MAE test 2.348851601244222
Epoch 9932 / 10000 loss: 11.757423162460327
MSE train 4.883783339268567 MSE test 11.289517095978038
MAE train 1.5138823752555182 MAE test 2.3488539975157843
Epoch 9933 / 10000 loss: 11.757082462310791
MSE train 4.8836811995019405 MSE test 11.289266273170808
MAE train 1.5138674470010032 MAE test 2.348821340753342
Epoch 9934 / 10000 loss: 11.75671648979187
MSE train 4.8835823289075835 MSE test 11.289287032777775
MAE train 1.5138469478766687 MAE test 2.348829546275604
Epoch 9935 / 10000 loss: 11.756415367126465
MSE train 4.88347903055766 MSE test 11.289044915756007
MAE train 1.5138315749461664 MAE test 2.3487980668697555
Epoch 9936 / 10000 loss: 11.755990982055664
MSE train 4.883384746289627 MSE test 11.289059133649868
MAE train 1.5138124099781827 MAE test 2.3488053714273023
Epoch 9937 / 10000 loss: 11.75567889213562
MSE train 4.883281788403778 MSE test 11.288800262799338
MAE train 1.5137974063297366 MAE test 2.348771654240353
Epoch 

Epoch 9985 / 10000 loss: 11.738343477249146
MSE train 4.878477672101516 MSE test 11.28326736495263
MAE train 1.5129531618421004 MAE test 2.348186758452712
Epoch 9986 / 10000 loss: 11.737944602966309
MSE train 4.878375169942313 MSE test 11.283290380612925
MAE train 1.5129316334640974 MAE test 2.3481953521915258
Epoch 9987 / 10000 loss: 11.737648010253906
MSE train 4.878271394148735 MSE test 11.283064810244953
MAE train 1.5129158369192983 MAE test 2.3481661370388704
Epoch 9988 / 10000 loss: 11.73720669746399
MSE train 4.878181171711689 MSE test 11.283066132509997
MAE train 1.5128980402974785 MAE test 2.348171772460914
Epoch 9989 / 10000 loss: 11.736879587173462
MSE train 4.878078557577325 MSE test 11.282795931225879
MAE train 1.5128833262377341 MAE test 2.348136580040107
Epoch 9990 / 10000 loss: 11.736502647399902
MSE train 4.877971323587326 MSE test 11.282821815401565
MAE train 1.512860465449957 MAE test 2.3481455994295786
Epoch 9991 / 10000 loss: 11.736215114593506
MSE train 4.87786817

In [ ]:
# pick a user
sample_user = X_test.iloc[99,:]
# get the predicted ratings
sample_user_pred = sess.run(output_layer, feed_dict={input_layer:[sample_user]})

In [25]:
# Save weights

v1 = sess.run(hidden_1_layer_vals)
f = open("weights.pkl","wb")
pickle.dump(v1,f)
f.close()

v2 = sess.run(output_layer_vals)
f = open("weights-output.pkl","wb")
pickle.dump(v2,f)
f.close()
